# Import

In [1]:
%install-swiftpm-flags -c release
%install '.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.6")' SwiftCoreMLTools
%install '.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")' Just

Installing packages:
	.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.6")
		SwiftCoreMLTools
	.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")
		Just
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmpxbchanhm/swift-install
Fetching https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git
Fetching https://github.com/dduan/Just.git
Fetching https://github.com/apple/swift-protobuf.git
Cloning https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git
Resolving https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git at 0.0.6
Cloning https://github.com/apple/swift-protobuf.git
Resolving https://github.com/apple/swift-protobuf.git at 1.8.0
Cloning https://github.com/dduan/Just.git
Resolving https://github.com/dduan/Just.git at 0.8.0
[1/3] Compiling Just Just.swift
[2/3] Compiling SwiftProtobuf AnyMessageStorage.swift
[3/4] Compiling SwiftCoreMLTools Activations.swift
[4/5] Compiling jupyterInstalledPackages jupyterInstalled

In [2]:
import Foundation
import SwiftCoreMLTools
import TensorFlow
import Just

# Data Download

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per ten thousand dollars
        - PTRATIO  pupil-teacher ratio by town
        - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
        - LSTAT    % lower status of the population
        - MEDV     Median value of owner-occupied homes in a thousand dollar

    :Missing Attribute Values: None

    :Creator: Harrison, D. and Rubinfeld, D.L.

This is a copy of UCI ML housing dataset.
https://archive.ics.uci.edu/ml/machine-learning-databases/housing/


In [3]:
// if let cts = Just.get(URL(string: "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data")!).content {
//     try! cts.write(to: URL(fileURLWithPath:"../data/housing.csv"))
// }

# Data Ingestion

In [4]:
let data = try String(contentsOfFile:"../data/housing.csv", encoding: String.Encoding.utf8)
let dataRecords: [[Float]] = data.split(separator: "\n").map{ String($0).split(separator: " ").compactMap{ Float(String($0)) } }

let numRecords = dataRecords.count
let numColumns = dataRecords[0].count

var index = Set<Int>()

while index.count < numRecords {
    index.insert(Int.random(in: 0..<numRecords))
}

let randomDataRecords = index.map{ dataRecords[$0] }

let dataFeatures = randomDataRecords.map{ Array($0[0..<numColumns-1]) }
let dataLabels = randomDataRecords.map{ Array($0[(numColumns-1)...]) }

# Data Transformation

## Split Numerical Categorical Features

In [5]:
let categoricalColumns = [3, 8]
let numericalColumns = [0, 1, 2, 4, 5, 6, 7, 9, 10, 11, 12]
let numCategoricalFeatures = categoricalColumns.count
let numNumericalFeatures = numericalColumns.count
let numLabels = 1

assert(numColumns == numCategoricalFeatures + numNumericalFeatures + 1)

// Get Categorical Features
let allCategoriesValues = dataFeatures.map{ row in categoricalColumns.map{ Int32(row[$0]) } }
                                .reduce(into: Array(repeating: [Int32](), count: 2)){ total, value in
                                    total[0].append(value[0])
                                    total[1].append(value[1]) }
                                .map{ Set($0).sorted() }

let categoricalFeatures = dataFeatures.map{ row in categoricalColumns.map{ Int32(row[$0]) } }

// Get Numerical Features
let numericalFeatures = dataFeatures.map{ row in numericalColumns.map{ row[$0] } }

## Categorize Categorical Features with Ordinal values

In [6]:
var categoricalValues = Array(repeating: Set<Int32>(), count: 2)

for record in categoricalFeatures {
    categoricalValues[0].insert(record[0])
    categoricalValues[1].insert(record[1])
}

let sortedCategoricalValues = [categoricalValues[0].sorted(), categoricalValues[1].sorted()]

let ordinalCategoricalFeatures = categoricalFeatures.map{ [Int32(sortedCategoricalValues[0].firstIndex(of:$0[0])!), 
                                                           Int32(sortedCategoricalValues[1].firstIndex(of:$0[1])!)] }

## Split Train and Test

In [7]:
let trainPercentage:Float = 0.8
let numTrainRecords = Int(ceil(Float(numRecords) * trainPercentage))
let numTestRecords = numRecords - numTrainRecords

func matrixTranspose<T>(_ matrix: [[T]]) -> [[T]] {
    if matrix.isEmpty {return matrix}
    var result = [[T]]()
    for index in 0..<matrix.first!.count {
        result.append(matrix.map{$0[index]})
    }
    return result
}

let xCategoricalAllTrain = matrixTranspose(Array(ordinalCategoricalFeatures[0..<numTrainRecords]))
let xCategoricalAllTest = matrixTranspose(Array(ordinalCategoricalFeatures[numTrainRecords...]))
let xNumericalAllTrain = Array(Array(numericalFeatures[0..<numTrainRecords]).joined())
let xNumericalAllTest = Array(Array(numericalFeatures[numTrainRecords...]).joined())
let yAllTrain = Array(Array(dataLabels[0..<numTrainRecords]).joined())
let yAllTest = Array(Array(dataLabels[numTrainRecords...]).joined())

let XCategoricalTrain = xCategoricalAllTrain.enumerated().map{ (offset, element) in 
    Tensor<Int32>(element).reshaped(to: TensorShape([numTrainRecords, 1]))
}
let XCategoricalTest = xCategoricalAllTest.enumerated().map{ (offset, element) in 
    Tensor<Int32>(element).reshaped(to: TensorShape([numTestRecords, 1]))
}

let XNumericalTrainDeNorm = Tensor<Float>(xNumericalAllTrain).reshaped(to: TensorShape([numTrainRecords, numNumericalFeatures]))
let XNumericalTestDeNorm = Tensor<Float>(xNumericalAllTest).reshaped(to: TensorShape([numTestRecords, numNumericalFeatures]))
let YTrain = Tensor<Float>(yAllTrain).reshaped(to: TensorShape([numTrainRecords, numLabels]))
let YTest = Tensor<Float>(yAllTest).reshaped(to: TensorShape([numTestRecords, numLabels]))

## Normalize Numerical Features

In [8]:
let mean = XNumericalTrainDeNorm.mean(alongAxes: 0)
let std = XNumericalTrainDeNorm.standardDeviation(alongAxes: 0)

print(mean, std)

[[ 3.568779,  11.25679, 11.137291, 0.5546997,   6.27381,   68.3178,  3.818542, 402.84445,
   18.41015, 358.97717,  12.61089]] [[   8.600502,   22.944748,   6.9644384, 0.117559254,  0.72180307,   28.166008,   2.0910063,
    168.27527,    2.141634,    88.84905,   7.2150693]]


In [9]:
let XNumericalTrain = (XNumericalTrainDeNorm - mean)/std
let XNumericalTest = (XNumericalTestDeNorm - mean)/std

In [10]:
print("Training shapes \(XNumericalTrain.shape) \(XCategoricalTrain[0].shape) \(XCategoricalTrain[1].shape) \(YTrain.shape)")
print("Testing shapes  \(XNumericalTest.shape) \(XCategoricalTest[0].shape) \(XCategoricalTest[1].shape) \(YTest.shape)")

Training shapes [405, 11] [405, 1] [405, 1] [405, 1]
Testing shapes  [101, 11] [101, 1] [101, 1] [101, 1]


# Model

In [11]:
struct MultiInputs<N: Differentiable, C>: Differentiable {
  var numerical: N
  
  @noDerivative
  var categorical: C

  @differentiable
  init(numerical: N, categorical: C) {
    self.numerical = numerical
    self.categorical = categorical
  }
}

struct RegressionModel: Module {
    var embedding1 = TensorFlow.Embedding<Float>(vocabularySize: 2, embeddingSize: 2)
    var embedding2 = TensorFlow.Embedding<Float>(vocabularySize: 9, embeddingSize: 5)
    var allInputConcatLayer = Dense<Float>(inputSize: (11 + 2 + 5), outputSize: 64, activation: relu)
    var hiddenLayer = Dense<Float>(inputSize: 64, outputSize: 32, activation: relu)
    var outputLayer = Dense<Float>(inputSize: 32, outputSize: 1)
    
    @differentiable
    func callAsFunction(_ input: MultiInputs<[Tensor<Float>], [Tensor<Int32>]>) -> Tensor<Float> {
        let embeddingOutput1 = embedding1(input.categorical[0])
        let embeddingOutput1Reshaped = embeddingOutput1.reshaped(to: 
            TensorShape([embeddingOutput1.shape[0], embeddingOutput1.shape[2]]))
        let embeddingOutput2 = embedding2(input.categorical[1])
        let embeddingOutput2Reshaped = embeddingOutput2.reshaped(to: 
            TensorShape([embeddingOutput2.shape[0], embeddingOutput2.shape[2]]))
        let allConcat = Tensor<Float>(concatenating: [input.numerical[0], embeddingOutput1Reshaped, embeddingOutput2Reshaped], alongAxis: 1)
        return allConcat.sequenced(through: allInputConcatLayer, hiddenLayer, outputLayer)
    }
}

var model = RegressionModel()

# Training

In [12]:
let optimizer = RMSProp(for: model, learningRate: 0.001)
Context.local.learningPhase = .training

In [13]:
let epochCount = 500
let batchSize = 32
let numberOfBatch = Int(ceil(Double(numTrainRecords) / Double(batchSize)))
let shuffle = true

func mae(predictions: Tensor<Float>, truths: Tensor<Float>) -> Float {
    return abs(Tensor<Float>(predictions - truths)).mean().scalarized()
}

In [14]:
for epoch in 1...epochCount {
    var epochLoss: Float = 0
    var epochMAE: Float = 0
    var batchCount: Int = 0
    var batchArray = Array(repeating: false, count: numberOfBatch)
    for batch in 0..<numberOfBatch {
        var r = batch
        if shuffle {
            while true {
                r = Int.random(in: 0..<numberOfBatch)
                if !batchArray[r] {
                    batchArray[r] = true
                    break
                }
            }
        }
        
        let batchStart = r * batchSize
        let batchEnd = min(numTrainRecords, batchStart + batchSize)
        let (loss, grad) = model.valueWithGradient { (model: RegressionModel) -> Tensor<Float> in
            let multiInput = MultiInputs(numerical: [XNumericalTrain[batchStart..<batchEnd]],
                                         categorical: [XCategoricalTrain[0][batchStart..<batchEnd],
                                                       XCategoricalTrain[1][batchStart..<batchEnd]])
            let logits = model(multiInput)
            return meanSquaredError(predicted: logits, expected: YTrain[batchStart..<batchEnd])
        }
        optimizer.update(&model, along: grad)
        
        let multiInput = MultiInputs(numerical: [XNumericalTrain[batchStart..<batchEnd]],
                                     categorical: [XCategoricalTrain[0][batchStart..<batchEnd],
                                                   XCategoricalTrain[1][batchStart..<batchEnd]])
        let logits = model(multiInput)
        epochMAE += mae(predictions: logits, truths: YTrain[batchStart..<batchEnd])
        epochLoss += loss.scalarized()
        batchCount += 1
    }
    epochMAE /= Float(batchCount)
    epochLoss /= Float(batchCount)

    print("Epoch \(epoch): MSE: \(epochLoss), MAE: \(epochMAE)")
}

Epoch 1: MSE: 555.85724, MAE: 21.593758
Epoch 2: MSE: 494.58957, MAE: 20.079403
Epoch 3: MSE: 417.35944, MAE: 18.056143
Epoch 4: MSE: 323.04916, MAE: 15.4796915
Epoch 5: MSE: 225.07655, MAE: 12.439347
Epoch 6: MSE: 141.13112, MAE: 9.17065
Epoch 7: MSE: 81.2541, MAE: 6.3849134
Epoch 8: MSE: 50.59609, MAE: 4.8456607
Epoch 9: MSE: 38.810886, MAE: 4.300005
Epoch 10: MSE: 32.426266, MAE: 3.9352217
Epoch 11: MSE: 28.155764, MAE: 3.6364465
Epoch 12: MSE: 24.99872, MAE: 3.4038403
Epoch 13: MSE: 22.808481, MAE: 3.258329
Epoch 14: MSE: 21.320107, MAE: 3.1203814
Epoch 15: MSE: 20.33838, MAE: 3.0329657
Epoch 16: MSE: 19.368086, MAE: 2.9625475
Epoch 17: MSE: 18.712317, MAE: 2.9187393
Epoch 18: MSE: 18.18215, MAE: 2.897068
Epoch 19: MSE: 17.590696, MAE: 2.840684
Epoch 20: MSE: 17.123863, MAE: 2.801013
Epoch 21: MSE: 16.708498, MAE: 2.787481
Epoch 22: MSE: 16.479849, MAE: 2.7366984
Epoch 23: MSE: 15.963711, MAE: 2.702882
Epoch 24: MSE: 15.681037, MAE: 2.6511755
Epoch 25: MSE: 15.3159075, MAE: 2.61998

Epoch 203: MSE: 4.0677505, MAE: 1.2310504
Epoch 204: MSE: 3.8893647, MAE: 1.2247624
Epoch 205: MSE: 3.8471096, MAE: 1.2164326
Epoch 206: MSE: 3.9869483, MAE: 1.2223322
Epoch 207: MSE: 3.7827413, MAE: 1.210011
Epoch 208: MSE: 3.9355226, MAE: 1.2213457
Epoch 209: MSE: 3.7930295, MAE: 1.2071694
Epoch 210: MSE: 3.8979528, MAE: 1.221236
Epoch 211: MSE: 3.8424923, MAE: 1.2112267
Epoch 212: MSE: 4.0192432, MAE: 1.206017
Epoch 213: MSE: 3.6404872, MAE: 1.2104465
Epoch 214: MSE: 3.7586617, MAE: 1.2035306
Epoch 215: MSE: 3.8147783, MAE: 1.1972021
Epoch 216: MSE: 3.6894784, MAE: 1.1882695
Epoch 217: MSE: 3.889833, MAE: 1.1893332
Epoch 218: MSE: 3.8685403, MAE: 1.1804007
Epoch 219: MSE: 3.660243, MAE: 1.176785
Epoch 220: MSE: 3.591872, MAE: 1.181987
Epoch 221: MSE: 3.616466, MAE: 1.1735991
Epoch 222: MSE: 3.589189, MAE: 1.1761128
Epoch 223: MSE: 3.6083422, MAE: 1.1791081
Epoch 224: MSE: 3.763592, MAE: 1.1755822
Epoch 225: MSE: 3.7189205, MAE: 1.1788684
Epoch 226: MSE: 3.614739, MAE: 1.1773599
Epoc

Epoch 401: MSE: 2.5129042, MAE: 0.89285874
Epoch 402: MSE: 2.432356, MAE: 0.8989935
Epoch 403: MSE: 2.391878, MAE: 0.9090339
Epoch 404: MSE: 2.382053, MAE: 0.9091116
Epoch 405: MSE: 2.568975, MAE: 0.9029995
Epoch 406: MSE: 2.3723063, MAE: 0.89014095
Epoch 407: MSE: 2.3531203, MAE: 0.91543454
Epoch 408: MSE: 2.2610624, MAE: 0.9059733
Epoch 409: MSE: 2.3803353, MAE: 0.8947563
Epoch 410: MSE: 2.3491175, MAE: 0.8957232
Epoch 411: MSE: 2.368956, MAE: 0.8814404
Epoch 412: MSE: 2.158277, MAE: 0.88086987
Epoch 413: MSE: 2.353748, MAE: 0.9367809
Epoch 414: MSE: 2.4315088, MAE: 0.90372974
Epoch 415: MSE: 2.366479, MAE: 0.888922
Epoch 416: MSE: 2.3756168, MAE: 0.8951221
Epoch 417: MSE: 2.2261536, MAE: 0.8750168
Epoch 418: MSE: 2.504664, MAE: 0.91973174
Epoch 419: MSE: 2.1368024, MAE: 0.87866914
Epoch 420: MSE: 2.2465808, MAE: 0.88729155
Epoch 421: MSE: 2.420235, MAE: 0.88819325
Epoch 422: MSE: 2.234766, MAE: 0.89463145
Epoch 423: MSE: 2.2678533, MAE: 0.893333
Epoch 424: MSE: 2.3503175, MAE: 0.890

# Test

In [15]:
Context.local.learningPhase = .inference

let multiInputTest = MultiInputs(numerical: [XNumericalTest],
                                 categorical: [XCategoricalTest[0],
                                               XCategoricalTest[1]])

let prediction = model(multiInputTest)

let predictionMse = meanSquaredError(predicted: prediction, expected: YTest).scalarized()/Float(numTestRecords)
let predictionMae = mae(predictions: prediction, truths: YTest)/Float(numTestRecords)

print("MSE: \(predictionMse), MAE: \(predictionMae)")

MSE: 0.10480358, MAE: 0.022593323


# Export

In [16]:
print(model.embedding1.embeddings.shape, model.embedding2.embeddings.shape)

[2, 2] [9, 5]


In [17]:
let coremlModel = Model(version: 4,
                        shortDescription: "Regression",
                        author: "Jacopo Mangiavacchi",
                        license: "MIT",
                        userDefined: ["SwiftCoremltoolsVersion" : "0.0.6"]) {
    Input(name: "numericalInput", shape: [11])
    Input(name: "categoricalInput1", shape: [1])
    Input(name: "categoricalInput2", shape: [1])
    Output(name: "output", shape: [1])
    NeuralNetwork {
        Embedding(name: "embedding1",
                     input: ["categoricalInput1"],
                     output: ["outEmbedding1"],
                     weight: model.embedding1.embeddings.transposed().flattened().scalars,
                     inputDim: 2,
                     outputChannels: 2)
        Permute(name: "permute1",
                     input: ["outEmbedding1"],
                     output: ["outPermute1"],
                     axis: [2, 1, 0, 3])
        Flatten(name: "flatten1",
                     input: ["outPermute1"],
                     output: ["outFlatten1"],
                     mode: .last)
        Embedding(name: "embedding2",
                     input: ["categoricalInput2"],
                     output: ["outEmbedding2"],
                     weight: model.embedding2.embeddings.transposed().flattened().scalars,
                     inputDim: 9,
                     outputChannels: 5)
        Permute(name: "permute2",
                     input: ["outEmbedding2"],
                     output: ["outPermute2"],
                     axis: [2, 1, 0, 3])
        Flatten(name: "flatten2",
                     input: ["outPermute2"],
                     output: ["outFlatten2"],
                     mode: .last)
        Concat(name: "concat",
                     input: ["numericalInput", "outFlatten1", "outFlatten2"],
                     output: ["outConcat"])
        InnerProduct(name: "dense1",
                     input: ["outConcat"],
                     output: ["outDense1"],
                     weight: model.allInputConcatLayer.weight.transposed().flattened().scalars,
                     bias: model.allInputConcatLayer.bias.flattened().scalars,
                     inputChannels: 11 + 2 + 5,
                     outputChannels: 64)
        ReLu(name: "Relu1",
             input: ["outDense1"],
             output: ["outRelu1"])
        InnerProduct(name: "dense2",
                     input: ["outRelu1"],
                     output: ["outDense2"],
                     weight: model.hiddenLayer.weight.transposed().flattened().scalars,
                     bias: model.hiddenLayer.bias.flattened().scalars,
                     inputChannels: 64,
                     outputChannels: 32)
        ReLu(name: "Relu2",
             input: ["outDense2"],
             output: ["outRelu2"])
        InnerProduct(name: "dense3",
                     input: ["outRelu2"],
                     output: ["output"],
                     weight: model.outputLayer.weight.transposed().flattened().scalars,
                     bias: model.outputLayer.bias.flattened().scalars,
                     inputChannels: 32,
                     outputChannels: 1)
    }
}

In [18]:
let coreMLData = coremlModel.coreMLData
try! coreMLData!.write(to: URL(fileURLWithPath: "../model/s4tf_house_simplified_trained_model.mlmodel"))

In [19]:
print(XNumericalTest[0], XCategoricalTest[0][0], XCategoricalTest[1][0], "=>", YTest[0])
print(XNumericalTest[17], XCategoricalTest[0][17], XCategoricalTest[1][17], "=>", YTest[17])
print(XNumericalTest[87], XCategoricalTest[0][87], XCategoricalTest[1][87], "=>", YTest[87])

[  -0.4000649,  -0.49060422,  -0.37006438,  -0.29516807,    0.2773472,     1.021877,
   -0.6626676,  -0.11198587,    1.1625932,   0.40813962, -0.047246937] [0] [4] => [19.8]
[   2.421373, -0.49060422,  0.99975175,   1.2359748,   -2.246887,    1.124838,  -1.1245025,
   1.5638397,   0.8357405,  0.42682302,     2.17172] [0] [8] => [10.5]
[-0.32639477,  0.38105497,  -1.0291269,   0.7851385,  -0.9889262, -0.19590291,  -0.8761533,
 -0.82510316,  -2.5261788,   0.3761753, -0.29949686] [0] [4] => [22.8]


In [33]:
Context.local.learningPhase = .inference

let numerical = [XNumericalTest[0].reshaped(to: TensorShape([1, 11]))]
let cat1 = XCategoricalTest[0][0].reshaped(to: TensorShape([1, 1]))
let cat2 = XCategoricalTest[1][0].reshaped(to: TensorShape([1, 1]))
let categorical = [cat1, cat2]

let multiInputTest = MultiInputs(numerical: numerical,
                                 categorical: categorical)

let prediction = model(multiInputTest)

print("0: \(prediction[0])")

0: [21.99591]


In [35]:
func predictTestRecord(_ record: Int) {
    Context.local.learningPhase = .inference

    let numerical = [XNumericalTest[record].reshaped(to: TensorShape([1, 11]))]
    let cat1 = XCategoricalTest[0][record].reshaped(to: TensorShape([1, 1]))
    let cat2 = XCategoricalTest[1][record].reshaped(to: TensorShape([1, 1]))
    let categorical = [cat1, cat2]

    let multiInputTest = MultiInputs(numerical: numerical,
                                     categorical: categorical)

    let prediction = model(multiInputTest)

    print("=== Test Record \(record) ===")
    print("Numerical Input: \(XNumericalTest[record])")
    print("Categorical Input: \(XCategoricalTest[0][record]) \(XCategoricalTest[1][record])")
    print("Prediction: \(prediction[0])")
    print("Y: \(YTest[record])")
    print("------------------")
}

In [36]:
predictTestRecord(0)
predictTestRecord(17)
predictTestRecord(87)

=== Test Record 0 ===
Numerical Input: [  -0.4000649,  -0.49060422,  -0.37006438,  -0.29516807,    0.2773472,     1.021877,
   -0.6626676,  -0.11198587,    1.1625932,   0.40813962, -0.047246937]
Categorical Input: [0] [4]
Prediction: [21.99591]
Y: [19.8]
------------------
=== Test Record 17 ===
Numerical Input: [   2.421373, -0.49060422,  0.99975175,   1.2359748,   -2.246887,    1.124838,  -1.1245025,
   1.5638397,   0.8357405,  0.42682302,     2.17172]
Categorical Input: [0] [8]
Prediction: [6.3390083]
Y: [10.5]
------------------
=== Test Record 87 ===
Numerical Input: [-0.32639477,  0.38105497,  -1.0291269,   0.7851385,  -0.9889262, -0.19590291,  -0.8761533,
 -0.82510316,  -2.5261788,   0.3761753, -0.29949686]
Categorical Input: [0] [4]
Prediction: [25.197638]
Y: [22.8]
------------------
